# **Top 3 Queries**
## **Medium Query**
**Proposition**: Returns a list of employees alongside their respective orders, including orders placed on specific numerical days.

**Database**: TSQLV4

**Table**: The query utilizes the HR.Employees table for employee information and the dbo.Orders table for order details.

**Columns**: It selects the employee's ID (empid), first name (firstname), and last name (lastname) from the HR.Employees table. Additionally, it retrieves the order ID (orderid) and order date (orderdate) from the dbo.Orders table.

**Predicate**: The query employs a JOIN operation to link the HR.Employees table with the dbo.Orders table based on the empid column. It also utilizes a LEFT JOIN to include all employees regardless of whether they have placed orders. Further, it joins with the dbo.Nums table, which likely serves as a sequence generator, to include orders placed on specific numerical days (from 1 to 5). 

*This is one of the greater queries because the BETWEEN operator is used here to establish a repitition of 5 rows of a result shown within a single row. Usually I have used the BETWEEN operator for filtration purposes, however, here for replication which is very interesting.*

In [ ]:
USE TSQLV4;

SELECT E.empid, E.firstname, E.lastname, O.orderid, O.orderdate
FROM HR.Employees AS E
    JOIN
    dbo.Nums AS N
    ON  N.n BETWEEN 1 AND 5
    LEFT JOIN
    dbo.Orders AS O
    ON 
    E.empid = O.empid
ORDER BY 
    E.empid, 
    N.n;
--FOR JSON PATH, ROOT('EmployeeOrderDetails');

## **Medium Query**
**Proposition**: Returns the customers with the highest total quantities specifically for customers located in the UK.

**Database**: TSQLV4

**Table**: The tables involved in this query are Sales.Customers, Sales.Orders, and Sales.OrderDetails.

**Columns**: Customer ID (custid): Identifies each customer uniquely.
Company Name (companyname): Name of the customer's company.
Number of Orders (num_orders): Count of distinct orders placed by each customer.
Total Quantity Ordered (total_quantity_ordered): Sum of quantities ordered across all orders by each customer.Average Unit Price (avg_unit_price): Average unit price of products ordered by each customer.

**Predicate**:The query utilizes INNER JOIN operations to link the Sales.Customers table with the Sales.Orders and Sales.OrderDetails tables based on the customer ID (custid) and order ID (orderid) columns, It filters the data to focus solely on customers located in the UK.The data is grouped by customer ID and company name to aggregate the metrics.
Finally, the results are ordered by the total quantity ordered in descending order to identify the customers with the highest total quantities ordered.

*This is one of the greater queries because this query retrieves detailed information about customers orders, usually more than other queries. This query utlizes two inner joins to retrieve this data and the result set is also restricted to orders only from the UK which is fascinating due to specificity.*

In [ ]:
USE TSQLV4;
SELECT
    C.custid,
    C.companyname,
    COUNT(DISTINCT O.orderid) AS num_orders,
    SUM(OD.qty) AS total_quantity_ordered,
    AVG(OD.unitprice) AS avg_unit_price
FROM
    Sales.Customers AS C
    INNER JOIN
    Sales.Orders AS O ON O.custid = C.custid
    INNER JOIN
    Sales.OrderDetails AS OD ON OD.orderid = O.orderid
WHERE 
    C.country = N'UK'
GROUP BY 
    C.custid, C.companyname
ORDER BY 
    total_quantity_ordered DESC;
--FOR JSON PATH, ROOT('CustomerOrderDetails');

## **Complex Query**
**Proposition**: Returns the total and average sales values for each employee

**Database**: Northwinds2022TSQLV7

**Table**: The query retrieves data from the  HumanResources.Employee table and the Sales.Order table. Also, this query involves a user-defined table-valued function named dbo.GetEmployeeSales, which computes sales by order for each employee.

**Columns**: The query uses columns from the HumanResources.Employee table, including EmployeeId, EmployeeLastName, and EmployeeFirstName. It also involves sales data, such as OrderId, UnitPrice, Quantity, and DiscountPercentage, from the Sales.Order and Sales.OrderDetail tables within a function named dbo.GetEmployeeSales. This function calculates total sales for each order. In the main query, these sales figures are summed up and averaged for each employee, resulting in two derived columns: TotalSalesValue and AverageOrderValue, which represent the sum and average of sales per employee, respectively.

**Predicate**: The dbo.GetEmployeeSales function calculates the total sales for each order by an employee, factoring in unit price, quantity, and discount. The main query then aggregates these results by employee, summing up total sales and calculating the average order value. An OUTER APPLY is used to match each employee with their sales, ensuring that employees with no sales are still included with a total sales value of 0. Also mention the use of aggregate functions SUM and AVG to calculate total and average sales values, respectively, and the OUTER APPLY clause to combine each employee with their respective sales data.

*This is a complex query of greater ranking because the result set is more compact with details and information, in comparison to the other complex queries.*


In [ ]:
USE Northwinds2022TSQLV7;

DROP FUNCTION IF EXISTS dbo.GetEmployeeSales;
GO

CREATE FUNCTION dbo.GetEmployeeSales(@EmployeeId AS INT)
RETURNS TABLE
AS
RETURN (
    SELECT
    O.EmployeeId,
    O.OrderId,
    SUM(OD.UnitPrice * OD.Quantity * (1 - OD.DiscountPercentage)) AS TotalSales
FROM Sales.[Order] O
    INNER JOIN Sales.OrderDetail OD ON O.OrderId = OD.OrderId
WHERE O.EmployeeId = @EmployeeId
GROUP BY O.EmployeeId, O.OrderId
);
GO

;WITH
    EmployeeSales
    AS
    (
        SELECT
            E.EmployeeId,
            E.EmployeeLastName,
            E.EmployeeFirstName,
            ISNULL(ES.TotalSales, 0) AS TotalSales
        FROM HumanResources.Employee E
    OUTER APPLY dbo.GetEmployeeSales(E.EmployeeId) ES
    )
SELECT
    ES.EmployeeId,
    ES.EmployeeLastName,
    ES.EmployeeFirstName,
    SUM(ES.TotalSales) AS TotalSalesValue,
    AVG(ES.TotalSales) AS AverageOrderValue
FROM EmployeeSales ES
GROUP BY ES.EmployeeId, ES.EmployeeLastName, ES.EmployeeFirstName
ORDER BY TotalSalesValue DESC;
--FOR JSON PATH, ROOT('EmployeeSales');


# **Top 3 Worst Queries**
## **Medium Query**
**Proposition**: Returns an analysis of orders placed by customers during the year 2000.

**Database**: Northwinds2022TSQLV7

**Table**: The query retrieves data from the Sales.Customer, Sales.Order, Sales.OrderDetail, and Production.Product tables.

**Columns**: It selects various pertinent columns including CustomerId, OrderId, ProductId, ProductName, and Quantity to provide comprehensive insights into customer orders and the products they purchased.

**Predicate**: The query employs JOIN operations to connect the Sales.Customer table with the Sales.Order and Sales.OrderDetail tables based on the CustomerId column, ensuring inclusion of all relevant order details for each customer. Then, it joins the Production.Product table to retrieve product names associated with each order detail. The WHERE clause filters the data to focus on orders placed in the year 2000 by using the YEAR function on the OrderDate column.

*This query is bad because the OrderDate column does not exist within the Sales.Customer table, thus, it cannot be joined with the Sales.Order table. This results to a lack of specificity as we are unable to filter the query to see the details of how many queries exist within a specified year, such as year 2000.*



In [ ]:
USE Northwinds2022TSQLV7;
SELECT C.CustomerId, O.OrderId, OD.ProductId, P.ProductName, OD.Quantity
FROM Sales.Customer AS C
  LEFT OUTER JOIN
  (Sales.[Order] AS O
  INNER JOIN Sales.OrderDetail AS OD
  ON O.OrderId = OD.OrderId)
  ON C.CustomerId = O.CustomerId
  INNER JOIN Production.Product AS P
  ON OD.ProductId = P.ProductId
WHERE YEAR(O.OrderDate) = 2000;

## **Medium Query**
**Proposition**: Returns a detailed analysis of orders placed by customers from Japan.

**Database**: TSQLV4

**Table**: The query retrieves data from the Sales.Customers, Sales.Orders, and Sales.OrderDetails tables.

**Columns**: It selects the Customer ID (custid) and Company Name (companyname) from the Sales.Customers table to uniquely identify each Japanese customer. Additionally, it calculates various metrics, including the number of unique orders (num_unique_orders), the count of ordered products (num_ordered_products), and the total order amount (total_order_amount) for each customer.

**Predicate**: The query utilizes INNER JOIN operations to link the Sales.Customers table with the Sales.Orders and Sales.OrderDetails tables based on the customer ID (custid) and order ID (orderid) columns. This ensures inclusion of only Japanese customers who have placed orders. It then filters the data to focus on customers from Japan by specifying 'JPN' as the country in the WHERE clause. Finally, it groups the results by customer ID and company name for aggregation.

*This query is bad because it cannot retrieve information regarding the Japanese customers who have placed orders. Information about the japanese customers who have placed orders have to be added in the Sales.Customers table.*

In [ ]:
USE TSQLV4;

SELECT
    C.custid,
    C.companyname,
    COUNT(DISTINCT O.orderid) AS num_unique_orders,
    COUNT(OD.productid) AS num_ordered_products,
    SUM(OD.qty * OD.unitprice) AS total_order_amount
FROM
    Sales.Customers AS C
    INNER JOIN
    Sales.Orders AS O ON O.custid = C.custid
    INNER JOIN
    Sales.OrderDetails AS OD ON OD.orderid = O.orderid
WHERE 
    C.country = N'JPN'
GROUP BY 
    C.custid,
    C.companyname;

## **Complex Query**
**Proposition**: Returns the total number of products and their average price provided by a specific supplier.

**Database**: Northwinds2022TSQLV7

**Table:** The query utilizes a user-defined table-valued function named dbo.GetProductsFromSupplier, which selects products based on the supplier ID provided. In addition, it retrieves data from the Production.Product table.

**Columns**: In the query provided, the columns used are from the Production.Product table and include ProductId, SupplierId, ProductName, and UnitPrice. These are accessed through a function named dbo.GetProductsFromSupplier, which filters products by a specific supplier. In the main query, the SupplierId is used to group the results, and two calculated columns are created: TotalProducts, which counts the number of products from the supplier, and AveragePrice, which calculates the average price of those products.

**Predicate**: The function dbo.GetProductsFromSupplier is applied to filter the products by a specific supplier ID. The main query then groups the results by SupplierId and calculates the total number of products supplied by this supplier and the average price of these products.
Also mention the aggregate functions COUNT and AVG used to calculate the total number of products and their average price, respectively.

*This query is not necessarily bad, however, it is reduced to a lower ranking due to the lack of detail the result set outputs. In addition declaring a parameter(@SupplierId)can be a more efficient way of doing executing this query*


In [ ]:
USE Northwinds2022TSQLV7;

DROP FUNCTION IF EXISTS dbo.GetProductsFromSupplier;
GO

CREATE FUNCTION dbo.GetProductsFromSupplier(@SupplierId INT)
RETURNS TABLE
AS
RETURN (
    SELECT
    P.ProductId,
    P.SupplierId,
    P.ProductName,
    P.UnitPrice
FROM Production.Product P
WHERE P.SupplierId = @SupplierId
);
GO
SELECT
    GPS.SupplierId,
    COUNT(GPS.ProductId) AS TotalProducts,
    AVG(GPS.UnitPrice) AS AveragePrice
FROM dbo.GetProductsFromSupplier(2) AS GPS
GROUP BY GPS.SupplierId;

# **Top 3 Fixed Queries**
## **Medium Query**
**Fixed:** Removed the filtration (OrderDate) which restricted the customer orders to the year 2000.

In [ ]:
USE Northwinds2022TSQLV7;

SELECT
    C.CustomerId,
    O.OrderId,
    OD.ProductId,
    P.ProductName,
    OD.Quantity
FROM
    Sales.Customer AS C
    LEFT JOIN Sales.[Order] AS O ON C.CustomerId = O.CustomerId
    INNER JOIN Sales.OrderDetail AS OD ON O.OrderId = OD.OrderId
    INNER JOIN Production.Product AS P ON OD.ProductId = P.ProductId;

## **Medium Query**
**Fixed:** Replaced the country as USA as it was Japan previously. The country USA is added to the Sales.Customers table.

In [ ]:
USE TSQLV4;

SELECT
    C.custid,
    C.companyname,
    COUNT(DISTINCT O.orderid) AS num_unique_orders,
    COUNT(OD.productid) AS num_ordered_products,
    SUM(OD.qty * OD.unitprice) AS total_order_amount
FROM
    Sales.Customers AS C
    INNER JOIN
    Sales.Orders AS O ON O.custid = C.custid
    INNER JOIN
    Sales.OrderDetails AS OD ON OD.orderid = O.orderid
WHERE 
    C.country = N'USA'
GROUP BY 
    C.custid,
    C.companyname;

--FOR JSON PATH, ROOT('EmployeeOrdersDetails');

## **Complex Query**
**Fixed:** There is no way to ultimately fix this code, however, this version of the code uses a parameter (better visual aspect) to select the specified supplier id, which is 2 in context to the query.

In [ ]:
USE Northwinds2022TSQLV7;

DROP FUNCTION IF EXISTS dbo.GetProductsFromSupplier;
GO

CREATE FUNCTION dbo.GetProductsFromSupplier(@SupplierId INT)
RETURNS TABLE
AS
RETURN (
    SELECT
    P.ProductId,
    P.SupplierId,
    P.ProductName,
    P.UnitPrice
FROM Production.Product P
WHERE P.SupplierId = @SupplierId
);
GO

DECLARE @SupplierId INT = 2;

SELECT
    GPS.SupplierId,
    COUNT(GPS.ProductId) AS TotalProducts,
    AVG(GPS.UnitPrice) AS AveragePrice
FROM dbo.GetProductsFromSupplier(@SupplierId) AS GPS
GROUP BY GPS.SupplierId;

--FOR JSON PATH, ROOT('ProductionProduct');

# **Medium Level Queries**
**Proposition**: Provides a summary of the total amount spent by each customer.

**Database**: WideWorldImporters

**Table**: The query retrieves data from the Sales.Orders and Sales.OrderLines tables.

**Columns**: It selects the CustomerID column from the Sales.Orders table to identify each customer uniquely. Additionally, it calculates the total spent by multiplying the quantity of each item (Quantity) with its unit price (UnitPrice) from the Sales.OrderLines table and sums up these values.

**Predicate**: The query employs an INNER JOIN operation to link the Sales.Orders table with the Sales.OrderLines table based on the orderid column. This join ensures that only orders with corresponding order lines are considered. It then groups the data by CustomerID to aggregate the total spent by each customer. Finally, it orders the results by CustomerID in ascending order.


In [ ]:
USE WideWorldImporters;

SELECT
    O.CustomerID,
    SUM(OD.Quantity * OD.UnitPrice) AS total_spent
FROM
    Sales.Orders AS O
    INNER JOIN
    Sales.OrderLines AS OD ON O.orderid = OD.orderid
GROUP BY 
    O.CustomerID
ORDER BY 
    O.CustomerID ASC;

**Proposition**: Returns a list of customers along with their corresponding country names, sorted by country name and customer name.

**Database**: PrestigeCars 

**Table**: The query involves three tables: Data.Customer, Data.Country, and Data.Sales.

**Columns**: The columns used are c.CustomerName for the name of the customer from the Data.Customer table, and co.CountryName for the name of the country from the Data.Country table.

**Predicate**: The query utilizes INNER JOIN clauses to relate the tables. The first INNER JOIN connects Data.Customer (aliased as c) with Data.Country (aliased as co) using the country code (c.Country matches co.CountryISO2), ensuring that only customers with a corresponding country are selected. The second INNER JOIN connects Data.Customer (c) with Data.Sales (s) using the CustomerID, which ensures that only customers who have made a sale are included in the result set.


In [ ]:
USE PrestigeCars;

SELECT
    c.CustomerName,
    co.CountryName
FROM
    [Data].[Customer] c
    INNER JOIN [Data].[Country] co ON c.Country = co.CountryISO2
    INNER JOIN [Data].[Sales] s ON c.CustomerID = s.CustomerID
ORDER BY
    co.CountryName,
    c.CustomerName;

**Proposition**: Returns details containing employee information, including their personal details, password hashes, and phone numbers.

**Database**: AdventureWorks2017

**Table**: The query retrieves data from the Person.Person, Person.Password, and Person.PersonPhone tables.

**Columns**: It selects the BusinessEntityID, FirstName, and LastName columns from the Person.Person table to represent each employee's unique identifier and name. Additionally, it includes the PasswordHash column from the Person.Password table to display the hashed passwords of employees. Furthermore, it incorporates the PhoneNumber column from the Person.PersonPhone table to present the contact numbers associated with each employee.

**Predicate**: The query employs JOIN operations to link the Person.Person table with the Person.Password and Person.PersonPhone tables based on the BusinessEntityID column, guaranteeing consistency of data across the tables. It then orders the results by BusinessEntityID in ascending order for clarity.




In [ ]:
USE AdventureWorks2017;

SELECT
    P.BusinessEntityID,
    P.FirstName,
    P.LastName,
    PS.PasswordHash,
    PP.PhoneNumber
FROM
    Person.Person AS P
    JOIN
    Person.Password AS PS ON P.BusinessEntityID = PS.BusinessEntityID
    INNER JOIN
    Person.PersonPhone AS PP ON P.BusinessEntityID = PP.BusinessEntityID
ORDER BY BusinessEntityID ASC;

**Proposition**: Returns details of the number of orders handled by each salesperson.

**Database**: WideWorldImportersDW

**Table**: The query retrieves data from the Fact.Order table.

**Columns**: It selects the Salesperson Key column to identify each salesperson uniquely. Additionally, it calculates the count of orders handled by each salesperson.

**Predicate**: The query employs LEFT OUTER JOIN operations to link the Fact.Order table with related fact tables (Fact.Purchase and Fact.Transaction) based on appropriate keys. This allows for inclusion of all orders regardless of whether they are associated with purchases or transactions. It then groups the data by Salesperson Key to aggregate the count of orders handled by each salesperson.


In [ ]:
USE WideWorldImportersDW;

SELECT
    O.[Salesperson Key],
    COUNT(*) AS Orders_Count
FROM
    Fact.[Order] AS O
    LEFT OUTER JOIN
    Fact.Purchase AS P ON P.[Date Key] = O.[Order Date Key]
    LEFT OUTER JOIN
    Fact.[Transaction] AS T ON T.[Lineage Key] = P.[Lineage Key]
GROUP BY
    O.[Salesperson Key];

**Proposition**: Returns an analysis of Canadian customer orders and their associated details.

**Database**: TSQLV4

**Table**: The query primarily focuses on data from the Sales.Customers, Sales.Orders, and Sales.OrderDetails tables.


**Columns**: It selects the Customer ID (custid) and Company Name (companyname) from the Sales.Customers table to uniquely identify each Canadian customer. Additionally, it calculates various metrics, including the number of unique orders (num_unique_orders), the count of ordered products (num_ordered_products), and the total order amount (total_order_amount) for each customer.

**Predicate**: The query utilizes LEFT JOIN operations to link the Sales.Customers table with the Sales.Orders and Sales.OrderDetails tables based on the customer ID (custid) and order ID (orderid) columns, respectively. This ensures inclusion of all Canadian customers, even if they haven't placed any orders. It then filters the data to focus solely on Canadian customers by specifying 'Canada' as the country in the WHERE clause. Finally, it groups the results by customer ID and company name for aggregation.


In [ ]:
USE TSQLV4;

SELECT
    C.custid,
    C.companyname,
    COUNT(DISTINCT O.orderid) AS num_unique_orders,
    COUNT(OD.productid) AS num_ordered_products,
    SUM(OD.qty * OD.unitprice) AS total_order_amount
FROM
    Sales.Customers AS C
    LEFT JOIN
    Sales.Orders AS O ON O.custid = C.custid
    LEFT JOIN
    Sales.OrderDetails AS OD ON OD.orderid = O.orderid
WHERE 
    C.country = N'Canada'
GROUP BY 
    C.custid,
    C.companyname;

**Proposition**: Returns a detailed analysis of customer orders and spending.

**Database**: Northwinds2022TSQLV7

**Table**: The query retrieves data from the Sales.Customer, Sales.Order, Sales.OrderDetail, and Production.Product tables.

**Columns**: It selects the CustomerId column to uniquely identify each customer. Additionally, it calculates various metrics, including the number of distinct orders (num_orders) and the total amount spent (total_spent) by each customer.

**Predicate**: The query utilizes INNER JOIN operations to link the Sales.Customer table with the Sales.Order and Sales.OrderDetail tables based on the CustomerId column. Additionally, it joins the Production.Product table to retrieve product information associated with each order detail. It then aggregates the data, calculating the count of distinct orders and the total amount spent by each customer, grouped by CustomerId.


In [ ]:
USE Northwinds2022TSQLV7;
SELECT
    C.CustomerId,
    COUNT(DISTINCT O.OrderId) AS num_orders,
    SUM(OD.Quantity * OD.UnitPrice) AS total_spent
FROM
    Sales.Customer AS C
    INNER JOIN
    Sales.[Order] AS O ON O.CustomerId = C.CustomerId
    INNER JOIN
    Sales.OrderDetail AS OD ON OD.OrderId = O.OrderId
    INNER JOIN
    Production.Product AS P ON OD.ProductId = P.ProductId
GROUP BY 
    C.CustomerId;

**Proposition**: Returns a detailed analysis of customer orders and spending habits, specifically focusing on Canadian customers.

**Database**: TSQLV4

**Table**: The query retrieves data from the Sales.Customers, Sales.Orders, Sales.OrderDetails, and HR.Employees tables.

**Columns**: It selects the Customer ID (custid) column to uniquely identify each customer. Additionally, it calculates various metrics, including the count of distinct orders (num_orders) and the total amount spent (total_amount_spent) by each customer.

**Predicate**: The query employs INNER JOIN operations to link the Sales.Customers table with the Sales.Orders and Sales.OrderDetails tables based on the customer ID (custid) and order ID (orderid) columns, respectively. It also joins the HR.Employees table to retrieve employee information associated with each order. The WHERE clause filters the data to focus solely on Canadian customers. Finally, it groups the results by customer ID and orders them by the number of orders in ascending order for clarity.


In [ ]:
USE TSQLV4;
SELECT
    C.custid,
    COUNT(DISTINCT O.orderid) AS num_orders,
    SUM(OD.qty * OD.unitprice) AS total_amount_spent
FROM
    Sales.Customers AS C
    INNER JOIN
    Sales.Orders AS O ON O.custid = C.custid
    INNER JOIN
    Sales.OrderDetails AS OD ON OD.orderid = O.orderid
    INNER JOIN
    HR.Employees AS E ON E.empid = O.empid
WHERE 
    C.country = N'Canada'
GROUP BY 
    C.custid
ORDER BY 
    num_orders ASC;

**Proposition**: Returns an analysis of customer orders and spending behavior, specifically focusing on customers from the USA.

**Database**: TSQLV4

**Table**: The query retrieves data from the Sales.Customers, Sales.Orders, Sales.OrderDetails, and HR.Employees tables.

**Columns**: It selects the Customer ID (custid) column to uniquely identify each customer. Additionally, it calculates various metrics, including the count of distinct orders (num_orders), the total amount spent (total_amount_spent), and the average unit price (avg_unit_price) of products purchased by each customer.

**Predicate**: The query employs INNER JOIN operations to link the Sales.Customers table with the Sales.Orders and Sales.OrderDetails tables based on the customer ID (custid) and order ID (orderid) columns, respectively. It also joins the HR.Employees table to retrieve employee information associated with each order. The WHERE clause filters the data to focus solely on customers from the USA. Finally, it groups the results by customer ID and orders them by customer ID for clarity.


In [ ]:
USE TSQLV4;
SELECT
    C.custid,
    COUNT(DISTINCT O.orderid) AS num_orders,
    SUM(OD.qty * OD.unitprice) AS total_amount_spent,
    AVG(OD.unitprice) AS avg_unit_price
FROM
    Sales.Customers AS C
    INNER JOIN
    Sales.Orders AS O ON O.custid = C.custid
    INNER JOIN
    Sales.OrderDetails AS OD ON OD.orderid = O.orderid
    INNER JOIN
    HR.Employees AS E ON E.empid = O.empid
WHERE 
    C.country = N'USA'
GROUP BY 
    C.custid
ORDER BY 
    C.custid;

**Proposition**: Returns customer orders and associated details, specifically focusing on customers from Mexico.

**Database**: TSQLV4

**Table**: The query primarily utilizes data from the Sales.Customers, Sales.Orders, and Sales.OrderDetails tables.

**Columns**: It selects the Customer ID (custid) and Company Name (companyname) columns to uniquely identify each customer. Additionally, it calculates various metrics, including the count of distinct orders (num_orders), the total quantity ordered (total_quantity_ordered), and the date of the first order (first_order_date) placed by each customer.

**Predicate**: The query employs INNER JOIN operations to link the Sales.Customers table with the Sales.Orders and Sales.OrderDetails tables based on the customer ID (custid) and order ID (orderid) columns, respectively. The WHERE clause filters the data to focus solely on customers from Mexico. Finally, it groups the results by customer ID and company name, ordering them by the number of orders in descending order for clarity.


In [ ]:
USE TSQLV4;
SELECT
    C.custid,
    C.companyname,
    COUNT(DISTINCT O.orderid) AS num_orders,
    SUM(OD.qty) AS total_quantity_ordered,
    MIN(O.orderdate) AS first_order_date
FROM
    Sales.Customers AS C
    INNER JOIN
    Sales.Orders AS O ON O.custid = C.custid
    INNER JOIN
    Sales.OrderDetails AS OD ON OD.orderid = O.orderid
WHERE 
    C.country = N'Mexico'
GROUP BY 
    C.custid, C.companyname
ORDER BY 
    num_orders DESC;

# **Complex Level Queries**
**Proposition**: Returns customer order details by product categories.

**Database**: Northwinds2022TSQLV7

**Table**: The query utilizes a user-defined table-valued function named dbo.GetCustomerOrders, which selects orders based on a given customer ID from the Sales.[Order] table.

**Columns**: The columns involved are directly from the Sales.[Order] table within the function: OrderId, CustomerId, EmployeeId, OrderDate, RequiredDate, ShipToRegion, ShipToPostalCode, ShipToCountry. In the main query, additional columns from related tables include Quantity from Sales.OrderDetail, and UnitPrice and CategoryId from Production.Product. The CategoryId is also related to the Production.Category table.

**Predicate**: The dbo.GetCustomerOrders function filters orders for a specific customer. The main query then joins these orders with order details and products, grouping the results by customer ID. It calculates the number of distinct product categories (NumCategories), the total quantity of products ordered (TotalQuantity), and the average price per product category (AvgPricePerCategory). Also mention the use of aggregate functions COUNT(DISTINCT ...), SUM, and AVG to compute the number of categories, total quantity, and average price per category, respectively.


In [ ]:
USE Northwinds2022TSQLV7;

DROP FUNCTION IF EXISTS dbo.GetCustomerOrders;
GO

CREATE FUNCTION dbo.GetCustomerOrders(@CustomerId AS INT)
RETURNS TABLE
AS
RETURN (
    SELECT
    OrderId,
    CustomerId,
    EmployeeId,
    OrderDate,
    RequiredDate,
    ShipToRegion,
    ShipToPostalCode,
    ShipToCountry
FROM Sales.[Order]
WHERE CustomerId = @CustomerId
);
GO
SELECT
    CO.CustomerId,
    COUNT(DISTINCT P.CategoryId) AS NumCategories,
    SUM(OD.Quantity) AS TotalQuantity,
    AVG(P.UnitPrice) AS AvgPricePerCategory
FROM dbo.GetCustomerOrders(6) AS CO
    INNER JOIN Sales.OrderDetail OD ON CO.OrderId = OD.OrderId
    INNER JOIN Production.Product P ON OD.ProductId = P.ProductId
    INNER JOIN Production.Category C ON P.CategoryId = C.CategoryId
GROUP BY CO.CustomerId;


**Proposition**: Analyzes the diversity and pricing of products supplied by a specific supplier

**Database**: Northwinds2022TSQLV7

**Table**: The query utilizes a user-defined table-valued function called dbo.GetProductsBySupplier, which retrieves products from the Production.Product table based on the supplied SupplierId.

**Columns**: Within the dbo.GetProductsBySupplier function, the columns used are ProductId, ProductName, SupplierId, CategoryId, and UnitPrice from the Production.Product table. The main query leverages SupplierId from the function's output and CategoryId for grouping products into categories.

**Predicate**: The function dbo.GetProductsBySupplier is designed to filter products by a given supplier. The main query then joins these products with the Production.Category table to ensure category data is accurate, subsequently grouping by SupplierId. It computes the number of distinct product categories (NumCategories) supplied by the supplier and the average unit price of these products (AvgPricePerProduct), offering insights into the range and average pricing of the supplier's products. Also mention the use of the COUNT(DISTINCT ...) function to determine the number of unique categories and AVG to calculate the average price per product.


In [ ]:
USE Northwinds2022TSQLV7;

DROP FUNCTION IF EXISTS dbo.GetProductsBySupplier;
GO

CREATE FUNCTION dbo.GetProductsBySupplier(@SupplierId AS INT)
RETURNS TABLE
AS
RETURN (
    SELECT
    ProductId,
    ProductName,
    SupplierId,
    CategoryId,
    UnitPrice
FROM Production.Product
WHERE SupplierId = @SupplierId
);
GO
SELECT
    PBS.SupplierId,
    COUNT(DISTINCT PBS.CategoryId) AS NumCategories,
    AVG(PBS.UnitPrice) AS AvgPricePerProduct
FROM dbo.GetProductsBySupplier(5) AS PBS
    INNER JOIN Production.Category C ON PBS.CategoryId = C.CategoryId
GROUP BY PBS.SupplierId;


**Proposition**: Returns employee performance in terms of customer reach, order volume, and average order value.

**Database**: Northwinds2022TSQLV7

**Table**: The query is built around the Sales.[Order] table, accessed through the dbo.GetOrdersByEmployee function, which filters orders handled by a specific employee.

**Columns**: Within the dbo.GetOrdersByEmployee function, the utilized columns are OrderId, CustomerId, EmployeeId, and OrderDate. In the main query, a subquery named OrderValue calculates TotalValue for each order, using UnitPrice, Quantity, and DiscountPercentage from the Sales.OrderDetail table.

**Predicate**: The dbo.GetOrdersByEmployee function serves to retrieve orders linked to a particular employee. The main query extends this by joining with a subquery that aggregates order details to compute the total value of each order. The result set is then grouped by EmployeeId, providing a count of unique customers (NumUniqueCustomers), a total count of orders (TotalOrders), and an average value of these orders (AvgOrderValue). 


In [ ]:
USE Northwinds2022TSQLV7;

DROP FUNCTION IF EXISTS dbo.GetOrdersByEmployee;
GO

CREATE FUNCTION dbo.GetOrdersByEmployee(@EmployeeId AS INT)
RETURNS TABLE
AS
RETURN (
    SELECT
    OrderId,
    CustomerId,
    EmployeeId,
    OrderDate
FROM Sales.[Order]
WHERE EmployeeId = @EmployeeId
);
GO
SELECT
    OBE.EmployeeId,
    COUNT(DISTINCT OBE.CustomerId) AS NumUniqueCustomers,
    COUNT(OBE.OrderId) AS TotalOrders,
    AVG(OrderValue.TotalValue) AS AvgOrderValue
FROM dbo.GetOrdersByEmployee(3) AS OBE
    INNER JOIN (
    SELECT
        OrderId,
        SUM(UnitPrice * Quantity * (1 - DiscountPercentage)) AS TotalValue
    FROM Sales.OrderDetail
    GROUP BY OrderId
) AS OrderValue ON OBE.OrderId = OrderValue.OrderId
GROUP BY OBE.EmployeeId;

**Proposition**: Returns shipping data for a specific product, including the number of shipments, total quantity shipped, and average quantity per shipment.

**Database**: Northwinds2022TSQLV7

**Table**: The query leverages a user-defined table-valued function named dbo.GetShipmentsByProduct, which retrieves shipment details from the Sales.OrderDetail and Sales.[Order] tables based on the provided ProductId.

**Columns**: Within the dbo.GetShipmentsByProduct function, the used columns are OrderId, ProductId, ShipToDate, and Quantity. The main query focuses on the ProductId to group the data, and then it calculates NumberOfShipments, TotalQuantityShipped, and AvgQuantityPerShipment as derived metrics.

**Predicate**: The dbo.GetShipmentsByProduct function is designed to filter shipments that include a specific product, excluding orders without a confirmed ship date. In the overarching query, the data is aggregated by ProductId, providing insights into the shipping activity for that product. This includes counting the total number of shipments (NumberOfShipments), summing up the total quantity of the product shipped (TotalQuantityShipped), and calculating the average quantity per shipment (AvgQuantityPerShipment). 


In [ ]:
USE Northwinds2022TSQLV7;

DROP FUNCTION IF EXISTS dbo.GetShipmentsByProduct;
GO

CREATE FUNCTION dbo.GetShipmentsByProduct(@ProductId AS INT)
RETURNS TABLE
AS
RETURN (
    SELECT
    OD.OrderId,
    OD.ProductId,
    O.ShipToDate,
    OD.Quantity
FROM Sales.OrderDetail OD
    INNER JOIN Sales.[Order] O ON OD.OrderId = O.OrderId
WHERE OD.ProductId = @ProductId AND O.ShipToDate IS NOT NULL
);
GO
SELECT
    SBP.ProductId,
    COUNT(SBP.OrderId) AS NumberOfShipments,
    SUM(SBP.Quantity) AS TotalQuantityShipped,
    AVG(SBP.Quantity) AS AvgQuantityPerShipment
FROM dbo.GetShipmentsByProduct(10) AS SBP
GROUP BY SBP.ProductId;


**Proposition**: Returns customer purchase history by totaling orders, revenue, and average order value.

**Database**: Northwinds2022TSQLV7

**Table**: The analysis is based on data pulled from the Sales.[Order] and Sales.OrderDetail tables through the dbo.GetOrdersByCustomer function, which isolates orders for a given customer.

**Columns**: Within the dbo.GetOrdersByCustomer function, the columns accessed are OrderId, CustomerId, OrderDate, Quantity, UnitPrice, and DiscountPercentage. The overarching query then employs CustomerId to group the data, while TotalOrders, TotalRevenue, and AvgOrderValue are calculated fields derived from the function's output.

**Predicate**: The dbo.GetOrdersByCustomer function is designed to filter orders specific to a customer, joining order and detail records to capture the quantity, unit price, and discounts of purchased items. The main query aggregates these details by customer ID, tallying the number of orders (TotalOrders), computing the total revenue (TotalRevenue), and determining the average value of these orders (AvgOrderValue). 


In [ ]:
USE Northwinds2022TSQLV7;

DROP FUNCTION IF EXISTS dbo.GetOrdersByCustomer;
GO

CREATE FUNCTION dbo.GetOrdersByCustomer(@CustomerId AS INT)
RETURNS TABLE
AS
RETURN (
    SELECT
    O.OrderId,
    O.CustomerId,
    O.OrderDate,
    OD.Quantity,
    OD.UnitPrice,
    OD.DiscountPercentage
FROM Sales.[Order] O
    INNER JOIN Sales.OrderDetail OD ON O.OrderId = OD.OrderId
WHERE O.CustomerId = @CustomerId
);
GO
SELECT
    GOC.CustomerId,
    COUNT(GOC.OrderId) AS TotalOrders,
    SUM(GOC.Quantity * GOC.UnitPrice * (1 - GOC.DiscountPercentage)) AS TotalRevenue,
    AVG(GOC.Quantity * GOC.UnitPrice * (1 - GOC.DiscountPercentage)) AS AvgOrderValue
FROM dbo.GetOrdersByCustomer(1) AS GOC
GROUP BY GOC.CustomerId;
